In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
!mkdir /content/gdrive/'My Drive'/model-training/

In [ ]:
%cd /content/gdrive/'My Drive'/model-training/

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

In [ ]:
%cd object_detection/

In [ ]:
DATASET_DIR = 'MicroobjectDataset/'

CLASS_1 = 'microobject'

In [ ]:
import glob
import xml.etree.ElementTree as ET

import tensorflow as tf

from utils import dataset_util


def create_record(name):
  tfrecord_writer = tf.io.TFRecordWriter(f"{DATASET_DIR}{name}.record")

  files = glob.glob(f"{DATASET_DIR}annotations/{name}/*.xml")

  files_n = len(files)
  counter = 1

  for xml_file in files:
    tree = ET.parse(xml_file)
    root = tree.getroot()

    filename = None
    width = None
    height = None

    class_texts = []
    class_labels = []

    x_mins = []
    y_mins = []
    x_maxes = []
    y_maxes = []

    for obj in root.findall('object'):
      filename = root.find('filename').text
      width = int(root.find('size').find('width').text)
      height = int(root.find('size').find('height').text)

      obj_class = obj.find('name').text.encode('utf-8')
      class_texts.append(obj_class)

      if obj_class == CLASS_1:
        class_labels.append(1)
      
      x_mins.append(int(obj.find('bndbox').find('xmin').text)/width)
      y_mins.append(int(obj.find('bndbox').find('ymin').text)/height)
      x_maxes.append(int(obj.find('bndbox').find('xmax').text)/width)
      y_maxes.append(int(obj.find('bndbox').find('ymax').text)/height) 

      with tf.io.gfile.GFile(f"{DATASET_DIR}images/{filename}", 'rb') as gfile:
        encoded_img = gfile.read()
      
      tf_example = tf.train.Example(features=tf.train.Features(feature={
          'image/height': dataset_util.int64_feature(height),
          'image/width': dataset_util.int64_feature(width),
          'image/filename': dataset_util.bytes_feature(filename.encode('utf8')),
          'image/source_id': dataset_util.bytes_feature(filename.encode('utf8')),
          'image/encoded': dataset_util.bytes_feature(encoded_img),
          'image/format': dataset_util.bytes_feature(b'jpg'),
          'image/object/bbox/xmin': dataset_util.float_list_feature(x_mins),
          'image/object/bbox/xmax': dataset_util.float_list_feature(x_maxes),
          'image/object/bbox/ymin': dataset_util.float_list_feature(y_mins),
          'image/object/bbox/ymax': dataset_util.float_list_feature(y_maxes),
          'image/object/class/text': dataset_util.bytes_list_feature(class_texts),
          'image/object/class/label': dataset_util.int64_list_feature(class_labels)
      }))
      tfrecord_writer.write(tf_example.SerializeToString())

      print(f"{name} record: {counter}/{files_n}")
      counter += 1
  
  tfrecord_writer.close()


create_record('train')
create_record('eval')

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
!tar -xf ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz --directory=MicroobjectDataset/

In [ ]:
!mkdir MicroobjectDataset/training/

In [ ]:
!python model_main_tf2.py --pipeline_config_path=MicroobjectDataset/ssd_mobilenet_v2_320x320_coco17_tpu-8/pipeline.config --model_dir=MicroobjectDataset/training/

In [ ]:
!pip install tensorboardcolab
%load_ext tensorboard
%tensorboard --logdir=MicroobjectDataset/training/

In [ ]:
!python model_main_tf2.py --pipeline_config_path=MicroobjectDataset/ssd_mobilenet_v2_320x320_coco17_tpu-8/pipeline.config --model_dir=MicroobjectDataset/training/ --checkpoint_dir=MicroobjectDataset/training/

In [ ]:
!mkdir MicroobjectDataset/tflite-model/

In [ ]:
!python export_tflite_graph_tf2.py --pipeline_config_path=MicroobjectDataset/ssd_mobilenet_v2_320x320_coco17_tpu-8/pipeline.config --trained_checkpoint_dir=MicroobjectDataset/training/ --output_directory=MicroobjectDataset/tflite-model/

In [ ]:
!tflite_convert --saved_model_dir=MicroobjectDataset/tflite-model/saved_model/ --output_file=MicroobjectDataset/tflite-model/model.tflite